In [1]:
import os

In [2]:
%pwd

'/Users/palakprashant/E2E_ML_Project_MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/palakprashant/E2E_ML_Project_MLflow'

In [12]:
#Prepare entity for model training
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_directory: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str



In [ ]:
#Create configuration manager
#Update configuration manager
from ml_proj.constants import *
from ml_proj.utils.common import read_yaml, create_directories
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = config_file,
            params_filepath = params_file,
            schema_filepath = schema_file):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_source])
    
    def get_model_trainer_configuration(self) -> ModelTrainerConfig:
        configuration = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        # Correct the call to create_directories
        create_directories([configuration.root_directory])

        model_trainer_configuration = ModelTrainerConfig(
            root_directory=configuration.root_directory,
            train_data_path=configuration.train_data_path,
            test_data_path=configuration.test_data_path,
            model_name = configuration.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name

        )
        return model_trainer_configuration

In [7]:
#Define components
import pandas as pd
import os
from ml_proj import logger
from sklearn.linear_model import ElasticNet
import joblib



In [ ]:
#Create components

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    def model_train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        training_x = train_data.drop([self.config.target_column], axis = 1)
        test_x = test_data.drop([self.config.target_column], axis = 1)

        training_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha = self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(training_x, training_y)


        joblib.dump(lr, os.path.join(self.config.root_directory, self.config.model_name))
        

In [13]:
try:
    config = ConfigurationManager()
    ModelTrainerConfiguration = config.get_model_trainer_configuration()
    ModelTrainerConfiguration = ModelTrainer(config = ModelTrainerConfiguration)
    ModelTrainerConfiguration.model_train()
except Exception as e:
    raise e

[2024-11-08 12:18:39,684: INFO: common: yaml file: src/ml_proj/config/config.yaml loaded successfully.]
[2024-11-08 12:18:39,699: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-11-08 12:18:39,706: INFO: common: yaml file: schema.yaml loaded successfully.]
[2024-11-08 12:18:39,707: INFO: common: Created directory here: artifacts]
[2024-11-08 12:18:39,707: INFO: common: Created directory here: artifacts/model_trainer]
